In [1]:
%matplotlib widget

In [2]:
from pysketcher import *

In [3]:
from ipywidgets import FloatSlider, AppLayout, Label, HBox

In [4]:
from IPython.display import SVG, display

In [5]:
myfig={}

In [6]:
gw = """\
libraries: ["from math import tan, radians, sin, cos","from pysketcher import *"]
fontsize: 18
g: 9.81              # constant gravity
theta: 30.0          # inclined plane angle
L: 10.0              # sketch sizing parameter
a: 1.0               #
xmin: 0.0            # sketech min Abscissa
ymin: -3.0           # sketech min Ordinate     
r: 1.0               # radius of the wheel
setframe:            # sketch setup
    action: "drawing_tool.set_coordinate_system(xmin=xmin, xmax=xmin+1.5*L,ymin=ymin, ymax=ymin+L+1,instruction_file='tmp_mpl.py')"
setblackline:        # default frame values and actions
    action: "drawing_tool.set_linecolor('black')"
B: point(a+L,0)                      # wall right end
A: point(a,tan(radians(theta))*L)    # wall left end
normal_vec: point(sin(radians(theta)),cos(radians(theta)))     # Vector normal to wall
tangent_vec: point(cos(radians(theta)),-sin(radians(theta)))   # Vector tangent to wall
help_line: Line(A,B)                 # wall line
x: a + 3*L/10.
y: help_line(x=x)    
contact: point(x, y)    
c: contact + r*normal_vec
outer_wheel: 
    formula: Circle(c, r)
    style:
        linecolor: blue
        filled_curves: blue
hole: 
    formula: Circle(c, r/2.)
    style:
        linecolor: blue
        filled_curves: white
N: 
    formula: Force(contact - 2*r*normal_vec, contact, r'$N$', text_pos='start')
    style:
        linecolor: black
mg: 
    formula: Gravity(c, 3*r, text='$Mg$')
    style:
        linecolor: black
wheel: 
    formula: "Composition({'outer': outer_wheel, 'inner': hole})"   
    style:
        shadow: 4
body: 
    formula: "Composition({'wheel': wheel, 'N': N, 'mg': mg})"
    style:
        linecolor: black
mB:
    formula: Text(r'$B$',B)
mA:
    formula: Text(r'$A$', A)
wall: 
    formula: Wall(x=[A[0], B[0]], y=[A[1], B[1]], thickness=-0.25,transparent=False)
    style:
        linecolor: black    
angle: 
    formula: "Arc_wText(r'$<bslash>theta$', center=B, radius=3, start_angle=180-theta, arc_angle=theta, fontsize=fontsize)"
    style:
        linecolor: black
        linewidth: 1
ground: 
     formula: Line((B[0]-L/10., 0), (B[0]-L/2.,0))
     stlye:
         linecolor: black
         linestyle: dashed
         linewidth: 1
x_const: 
    formula: Line(contact, contact + point(0,4))
    style:
        linestyle: dotted
    transform: rotate(-theta, contact)
x_axis: 
    formula: "Axis(start=contact+ 3*r*normal_vec, length=4*r,label='$x$', rotation_angle=-theta)"
fixed: 
    formula: "Composition({'angle': angle, 'inclined wall': wall,'ground': ground,'x start': x_const, 'x axis': x_axis, 'mA': mA, 'mB': mB})"
fig: 
    formula: "Composition({'body': body, 'fixed elements': fixed})"
"""

In [7]:
sketchParse(gw,myfig)

In [8]:
d = myfig['fig'].draw()

In [9]:
drawing_tool.display()

In [10]:
def position(t):
    """Position of center point of wheel."""
    global tangent_vec,c,myfig
    return myfig['c'] + 0.5*myfig['g']*t**2*myfig['tangent_vec']
t = 0
def move(change):
    global fig,t,label,x,x0,dt
    dt = change.new - t 
    t = change.new
    drawing_tool.erase()
    x = position(t)
    x0 = position(t-dt)
    displacement = x - x0    
    myfig['fig']['body'].translate(displacement)
    myfig['fig'].draw()
    label.value = f"p:{x}"

In [11]:
slider = FloatSlider(
    orientation='horizontal',
    description='Time:',
    value=0.0,
    min=0.0,
    max=1.0,
    step = 1.0 / 30
)
slider.observe(move, 'value')

In [12]:
label = Label(value="")

In [13]:
AppLayout(
    center=drawing_tool.mpl.gcf().canvas,
    footer=HBox([slider,label]),
    pane_heights=[0, 6, 1]
)
#drawing_tool.mpl.ion()

AppLayout(children=(HBox(children=(FloatSlider(value=0.0, description='Time:', max=1.0, step=0.033333333333333…

In [14]:
drawing_tool.mpl.ion()

In [15]:
myfig['fig'].graphviz_dot('fig')

graphviz [('Composition:\\nfig', 'Composition:\\nbody'), ('Composition:\\nbody', 'Composition:\\nwheel'), ('Composition:\\nwheel', 'Circle:\\nouter'), ('Circle:\\nouter', 'Curve:\\narc'), ('Composition:\\nwheel', 'Circle:\\ninner'), ('Circle:\\ninner', 'Curve:\\narc'), ('Composition:\\nbody', 'Force:\\nN'), ('Force:\\nN', 'Line:\\narrow'), ('Line:\\narrow', 'Curve:\\nline'), ('Force:\\nN', 'Text:\\ntext'), ('Composition:\\nbody', 'Gravity:\\nmg'), ('Gravity:\\nmg', 'Line:\\narrow'), ('Line:\\narrow', 'Curve:\\nline'), ('Gravity:\\nmg', 'Text:\\ntext'), ('Composition:\\nfig', 'Composition:\\nfixed elements'), ('Composition:\\nfixed elements', 'Arc_wText:\\nangle'), ('Arc_wText:\\nangle', 'Arc:\\narc'), ('Arc:\\narc', 'Curve:\\narc'), ('Arc_wText:\\nangle', 'Text:\\ntext'), ('Composition:\\nfixed elements', 'Wall:\\ninclined wall'), ('Wall:\\ninclined wall', 'Curve:\\nwall'), ('Composition:\\nfixed elements', 'Line:\\nground'), ('Line:\\nground', 'Curve:\\nline'), ('Composition:\\nfixed 

In [ ]:
!dot -Tpng -o fig.png fig.dot

![](fig.png)

In [ ]:
drawing_tool.mpl.gcf().canvas.print_png("fig1.png")

![](fig1.png)

In [ ]:
display(SVG(sketch2SVG()))